# Observado Ajustado con Climatogía Observada - Lluvia(Diario)

## Cargar librerias
Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [10]:
import glob # Para buscar archivos en un directorio.
import pandas as pd # Para manipulación de datos.
import matplotlib.pyplot as plt # Para visualización de datos en 2D.
import numpy as np # Para cálculos numéricos.
from datetime import datetime, date, timedelta # Para fechas y horas.
import requests # Para realizar solicitudes HTTP.
import urllib.request # Para trabajar con URLs.
import io # Para manejar entradas/salidas.
import os # Para interactuar con el sistema operativo.

Definir parametros para usar más adelante

In [11]:
# Diccionario que asocia números de meses con sus nombres en español
meses_espanol = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio", 7: "julio",
    8: "agosto", 9: "setiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

fecha = date.today()
file_date = fecha.strftime("%Y%m%d")  # Agregar la fecha en formato YYYYMMDD a file_date

# Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]

## Cargar y transformar datos desde enlace de lluvia observada

In [12]:
# Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
url_archivo = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/acumuladosMinutos.txt"

# Verificar si el archivo de lluvia observada existe antes de descargarlo
response = requests.get(url_archivo)

if response.status_code == 200: #Archivo existe: código de estado HTTP 200
    dat_lluvia = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
    
    # Dividir el contenido del archivo en líneas usando "\r\n" como separador
    lineas = dat_lluvia.split("\r\n")

    # Inicializar una lista para almacenar los datos de las estaciones de este archivo
    estaciones = []

    # Variable para determinar si hemos encontrado la sección de una zona geográfica
    seccion_encontrada = False
    
    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        if "******" in linea:
            seccion_encontrada = True
        elif "..." in linea:
            seccion_encontrada = True
        elif seccion_encontrada and "-" in linea:
            datos_estacion = linea.split("  ")

            # Eliminar líneas vacías antes de procesar los datos
            datos_estacion = [dato.strip() for dato in datos_estacion if dato.strip()]

            # Asegurarnos de que la línea no esté vacía después de la eliminación
            if datos_estacion and not datos_estacion[0].startswith("-----"):
                estacion = datos_estacion[0].split("-")[0]
                acumulado_7am = datos_estacion[3].split(" ")[0]  # Corregir el índice para obtener el acumulado desde las 7 am

                # Solo usar las lineas donde lluvia024 es diferente a 'NA'
                if acumulado_7am != "NA": 
                    estacion = int(estacion) # Convertir el valor a tipo int
                    acumulado_7am = float(acumulado_7am) # Convertir el valor a tipo float64

                    # Agregar los datos de la estación y lluvia a la lista estaciones
                    estaciones.append((estacion, acumulado_7am))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    lluvia_observada = pd.DataFrame(estaciones, columns=["Estacion", "Lluvia"])
else:
    #Imprimir un mensaje indicando que para la fecha indicada el archivo de lluvia observada correspondiente no existe
    print(f"El archivo de la {file_date} no existe en la URL: {url_archivo}")

## Cargar y transformar datos desde enlace de temperatura observada

In [13]:
# Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
url_archivo = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/temperaturasActuales.txt"

# Verificar si el archivo de lluvia observada existe antes de descargarlo
response = requests.get(url_archivo)

if response.status_code == 200: #Archivo existe: código de estado HTTP 200
    dat_temp = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
    
    # Dividir el contenido del archivo en líneas usando "\n" como separador
    lineas = dat_temp.strip().split("\n")[1:]  # Omitir la primera línea que es la cabecera

    # Inicializar una lista para almacenar los datos de las estaciones de este archivo
    estaciones = []

    # Variable para determinar si hemos encontrado la sección con estaciones
    seccion_encontrada = False

    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        if "*****" in linea:
            seccion_encontrada = True
        elif "....." in linea:
            seccion_encontrada = True
        elif seccion_encontrada and "-" in linea:
            datos_estacion = linea.split("   ")

            # Eliminar líneas vacías antes de procesar los datos
            datos_estacion = [dato.strip() for dato in datos_estacion if dato.strip()]

            # Asegurarnos de que la línea no esté vacía después de la eliminación
            if datos_estacion and not datos_estacion[0].startswith("-----"):
                estacion = datos_estacion[0].split("-")[0]
                tmax = datos_estacion[4]  # Corregir el índice para obtener la longitud
                tmin = datos_estacion[5]   # Corregir el índice para obtener la latitud
                
                # Solo usar las lineas donde tmax, tmin es diferente a 'NA'
                if tmax != "NA" and tmin != "NA":
                    tmax = float(tmax)# Convertir el valor a tipo float64
                    tmin = float(tmin)# Convertir el valor a tipo float64
                    estacion = int(estacion)# Convertir el valor a tipo int
                    
                    # Agregar los datos de la estación, tmax y tmin a la lista estaciones
                    estaciones.append((estacion, tmax, tmin))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    temp_observada = pd.DataFrame(estaciones, columns=["Estacion", "Tmax", "Tmin"])
else:
    #Imprimir un mensaje indicando que para la fecha indicada el archivo de lluvia observada correspondiente no existe
    print(f"El archivo de la {file_date} no existe en la URL: {url_archivo}")

## Cargar los valores extremos de la climatologia mensual de lluvia observada 

In [14]:
# Ruta del archivo Excel que contiene los extremos mensuales
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Lluvia/Climatologia_Mensual_Observada_Lluvia.xlsx'

# Lista de nombres de las hojas de Excel
hojas_excel = ['min', 'max']

# Cargar las tablas y convertir los nombres de las columnas a cadenas de texto
tablas_lluvia = {}
for hoja in hojas_excel:
    tabla = pd.read_excel(archivo_excel, sheet_name=hoja)
    tabla.columns = tabla.columns.astype(str)
    tablas_lluvia[hoja] = tabla

## Cargar los valores extremos de la climatologia mensual de temperatura observada 

In [15]:
# Ruta del archivo Excel
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Temperatura/Climatologia_Mensual_Observada_Temperatura.xlsx'

# Lista de nombres de las hojas de Excel
hojas_excel = ['tmax_min', 'tmax_max', 'tmin_min', 'tmin_max']

# Cargar las tablas y convertir los nombres de las columnas a cadenas de texto
tablas_temp = {}
for hoja in hojas_excel:
    tabla = pd.read_excel(archivo_excel, sheet_name=hoja)
    tabla.columns = tabla.columns.astype(str)
    tablas_temp[hoja] = tabla

## Agregar valores a tabla de ajuste

In [16]:
# Crear un diccionario vacío
tabla_ajuste_data_observada = {}

# Asignar las columnas a partir de la columna 1 en adelante a la clave 'Estacion' del diccionario
tabla_ajuste_data_observada['Estacion'] = tablas_lluvia['min'].iloc[:, 1:].columns.tolist()

# Crear un DataFrame a partir del diccionario
tabla_ajuste_data_observada = pd.DataFrame(tabla_ajuste_data_observada)

# Iterar a través de las estaciones en la columna "Estacion" de tabla_ajuste_data_observada
for estacion in tabla_ajuste_data_observada['Estacion']:
    estacion = str(estacion)
    
    try:
        clim_lluvia_min = tablas_lluvia['min'].loc[tablas_lluvia['min']["mes"] == mes, estacion].values[0]
        clim_lluvia_max = tablas_lluvia['max'].loc[tablas_lluvia['max']["mes"] == mes, estacion].values[0]

        clim_tmax_min = tablas_temp['tmax_min'].loc[tablas_temp['tmax_min']["FECHA"] == mes, estacion].values[0]
        clim_tmax_max = tablas_temp['tmax_max'].loc[tablas_temp['tmax_max']["FECHA"] == mes, estacion].values[0]

        clim_tmin_min = tablas_temp['tmin_min'].loc[tablas_temp['tmin_min']["FECHA"] == mes, estacion].values[0]
        clim_tmin_max = tablas_temp['tmin_max'].loc[tablas_temp['tmin_max']["FECHA"] == mes, estacion].values[0]
        
        filas_lluvia = lluvia_observada.loc[lluvia_observada["Estacion"].astype(str).str.strip() == estacion.strip()]
        filas_temp = temp_observada.loc[temp_observada["Estacion"].astype(str) == estacion]

        if not filas_lluvia.empty:
            diario_lluvia = filas_lluvia["Lluvia"].iloc[0]
            
            if diario_lluvia > clim_lluvia_max:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Lluvia'] = f"{clim_lluvia_max}({diario_lluvia})"
            elif diario_lluvia < 0:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Lluvia'] = f"{clim_lluvia_min}({diario_lluvia})"
            else:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Lluvia'] = "-"
        else:
            tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Lluvia'] = "x"

        if not filas_temp.empty:
            diario_tmax = filas_temp["Tmax"].iloc[0]
            diario_tmin = filas_temp["Tmin"].iloc[0]

            if diario_tmax > clim_tmax_max:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmax'] = f"{clim_tmax_max}({diario_tmax})"
            elif diario_tmax < clim_tmax_min:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmax'] = f"{clim_tmax_min}({diario_tmax})"
            else:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmax'] = "-"

            if diario_tmin > clim_tmin_max:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmin'] = f"{clim_tmin_max}({diario_tmin})"
            elif diario_tmin < clim_tmin_min:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmin'] = f"{clim_tmin_min}({diario_tmin})"
            else:
                tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmin'] = "-"
        else:
            tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmax'] = "x"
            tabla_ajuste_data_observada.loc[tabla_ajuste_data_observada['Estacion'] == estacion, 'Tmin'] = "x"
            
    except KeyError:
        pass
        print(f'{estacion} no existe en tablas_temp o tablas_lluvia')
        
        
# Ruta de la carpeta donde se almacenará los datos procesados
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Obs_ajuste_w_Clim"
        
# Guarda el DataFrame del ajuste del pronostico de mañana en archivos CSV y TXT
tabla_ajuste_data_observada.to_csv(f"{carpeta}/tabla_ajuste_data_observada.csv", index=False)
tabla_ajuste_data_observada.to_csv(os.path.join(carpeta, "tabla_ajuste_data_observada.txt"), index=False)

# Guarda el DataFrame del ajuste del pronostico de mañana en archivos CSV y TXT
tabla_ajuste_data_observada.to_csv(f"{carpeta}/tabla_ajuste_data_observada_{file_date}.csv", index=False)
tabla_ajuste_data_observada.to_csv(os.path.join(carpeta, f"tabla_ajuste_data_observada_{file_date}.txt"), index=False)

72193 no existe en tablas_temp o tablas_lluvia


In [17]:
tabla_ajuste_data_observada

,Estacion,Lluvia,Tmax,Tmin
0,100633,-,-,-
1,100641,-,-,-
2,100643,-,-,-
3,100649,-,-,-
4,100651,-,-,-
5,100653,-,-,-
6,100655,-,-,-
7,200002,x,x,x
8,200007,x,x,x
9,200009,x,x,x


In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(tabla_ajuste_data_observada)

    Estacion Lluvia        Tmax        Tmin
0     100633      -           -           -
1     100641      -           -           -
2     100643      -           -           -
3     100649      -           -           -
4     100651      -           -           -
5     100653      -           -           -
6     100655      -           -           -
7     200002      x           x           x
8     200007      x           x           x
9     200009      x           x           x
10     69633      -           x           x
11     69647      -           -           -
12     69677      x           -           -
13     69679      -           -           -
14     69681      -           x           x
15     69699      -           x           x
16     69701      -           -           -
17     69709      -           -           -
18     69711      -           -           -
19     69713      -           -           -
20     69715      -           -           -
21     69717      -           - 